# Setup

In [ ]:
import numpy as np
from textblob import TextBlob
import nltk
import pandas as pd
import matplotlib as plt
import seaborn as sn

# ANN - Ethan
# SVM - Lily
# RF - Marinana
# Adaboost - Qingru

# Source: https://www.kaggle.com/datasets/radheshyamkollipara/bank-customer-churn/data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Helper Functions

# splice and reform data between categorical, dummy categorical, and numeric
def spliceData(cleandata_df, categorical_columns, dummy_categorical_columns, numeric_columns, target_columns):
    # splice out columns
    categorical_df = cleandata_df[categorical_columns]
    dummy_categorical_df = cleandata_df[dummy_categorical_columns]
    numeric_df = cleandata_df[numeric_columns]
    ##TODO: Scale numerical data
    # Make dummies
    dummy_code_cat_var = pd.get_dummies(dummy_categorical_df, drop_first=False, dtype='uint8')

    df_predictor = pd.concat([categorical_df, numeric_df, dummy_code_cat_var], axis=1)
    target = cleandata_df[target_columns]
    return target, df_predictor


# Split the data into test and train
from sklearn.model_selection import train_test_split
def splitData(predictor, target, size):
    x_train, x_test, y_train, y_test = train_test_split(predictor, target, test_size=size)
    return x_train, x_test, y_train, y_test

# Show Classification, Confusion Matrix, and ROC AUC
def display_triad(model, x_test_scale, y_test, matrix_title):
    y_pred = model.predict(x_test_scale)
    display_classification_report(y_test, y_pred)

    display_confusion_matrix(y_test, y_pred, matrix_title)

    y_prob_pred = model.predict_proba(x_test_scale)[:,1]
    display_auc_roc(y_test, y_prob_pred)

# Show ROC AUC curve
from sklearn import metrics
def display_auc_roc(y_test, prob_pred):
    fpr, tpr, t = metrics.roc_curve(y_test, prob_pred)
    auc = metrics.roc_auc_score(y_test, prob_pred)
    plt.plot(fpr, tpr, label = "AUC="+str(100*round(auc,3))+"%")
    plt.legend()
    plt.show()


# Show Classification Report
from sklearn.metrics import classification_report
def display_classification_report(y_test, y_pred):
    # Predict a value from the model
    print(classification_report(y_test, y_pred))

# Show Confusion Matrix
from sklearn.metrics import confusion_matrix
def display_confusion_matrix(y_test, y_pred, title):
    cnf_matrix = confusion_matrix(y_test, y_pred)
    # cnf_matrix
    cf_df = pd.DataFrame(cnf_matrix, columns=['0', '1'], index=['0', '1'])
    plt.figure(figsize=(7,5))
    sn.heatmap(cf_df, annot=True, fmt='g')
    plt.title(title)
    plt.xlabel('Predictor')
    plt.ylabel('Actual')
    plt.show()

In [ ]:
# Can we determine if a customer will leave the bank, so that the bank can offer new promotions to keep that customer?
# Since its more expensive to sign on a new customer than to keep

churn = pd.read_csv("Customer-Churn-Records.csv")
churn.columns

# Take out 'RowNumber', 'CustomerId', 'Surname' - Useless
# take out 'Complain', 'Satisfaction Score' - Endogenous

In [ ]:
# Calculate churn rate for each age group
churn['Age_Group'] = pd.cut(churn['Age'], bins=range(0, 101, 10), right=False)

# Calculate churn rate for each age group
churn_rate_per_age_group = churn.groupby('Age_Group')['Exited'].mean()

# Plotting
plt.figure(figsize=(10, 6))
churn_rate_per_age_group.plot(kind='bar')
plt.title('Churn Rate by Age Group',fontsize = 16)
plt.xlabel('Age Group',fontsize = 14)
plt.ylabel('Churn Rate',fontsize = 14)
plt.xticks(rotation=45,fontsize = 12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x='Geography', hue='Exited', data=churn)
plt.title('Churn by Geography')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
# Create a pie chart of the churn distribution
labels = ["Not Exited", "Exited"]
sizes = [len(churn[churn['Exited'] == 0]), len(churn[churn['Exited'] == 1])]
colors = ["green", "lightcoral"]

plt.pie(sizes, labels=labels, autopct="%1.1f%%", colors=colors)
plt.title("Churn Distribution")
plt.show()


# Cleaning

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#  'Geography', 'Card Type','Gender',
cols = ['CreditScore',
       'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Point Earned']


# Create heatmap
plt.figure(figsize=(10, 6))  # Set the figure size
sns.heatmap(churn[cols].corr(), annot=True)  # cmap is the color map
plt.title('Heatmap of DataFrame')  # Set title
# plt.xlabel('Columns')  # Set x-axis label
# plt.ylabel('Index')  # Set y-axis label
plt.show()


In [ ]:
# Steps, normalize, scale, and clean data.
# Class balance, exists an 80/20 split in class
# Incredibly, there are no missing values at all.
using = ['CreditScore',
       'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'Geography', 'Card Type', 'Gender',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Point Earned']
churn[using].info()

In [ ]:
# Dummify and Scale
cat_list = []
numeric_list = ['CreditScore',
       'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Point Earned']
dummy_list = ['Geography', 'Card Type', 'Gender', ]
target_list = ['Exited']

target_df, predictor_df = spliceData(churn, cat_list, dummy_list, numeric_list, target_list)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(predictor_df[numeric_list], target_df)
predictor_df_scale = pd.DataFrame(scaler.transform(predictor_df[numeric_list]))

predictor_df_scale.columns = predictor_df[numeric_list].columns.values
predictor_df_scale.index = predictor_df[numeric_list].index.values

predictor_df[numeric_list] = predictor_df_scale

In [ ]:
# Class Balancing
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1000)

predictor_df, target_df = smote.fit_resample(predictor_df, target_df)

In [ ]:
# Train Test Split
x_train, x_test, y_train, y_test = splitData(predictor_df, target_df, size=0.3)

In [ ]:
import matplotlib.pyplot as plt
labels = ["Not Exited", "Exited"]
sizes_train = [len(y_train[y_train['Exited'] == 0]), len(y_train[y_train['Exited'] == 1])]
sizes_test = [len(y_test[y_test['Exited'] == 0]), len(y_test[y_test['Exited'] == 1])]

fig, axs = plt.subplots(1, 2)

axs[0].pie(sizes_train, labels=labels, autopct="%1.1f%%", colors=["green", "lightcoral"])
axs[0].set_title("Train Set")

axs[1].pie(sizes_test, labels=labels, autopct="%1.1f%%", colors=["green", "lightcoral"])
axs[1].set_title("Test Set")

plt.show()

#ANN

In [ ]:
# Setup ANN
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import regularizers

# Ann - Basic
print(x_train.shape)

model = Sequential()
model.add(Dense(20,input_shape=(18,),activation="sigmoid"))
model.add(Dropout(0.2))
model.add(Dense(20, activation="selu", kernel_regularizer=regularizers.l2()))
model.add(Dropout(0.2))
model.add(Dense(20, activation="selu", kernel_regularizer=regularizers.l2()))
model.add(Dropout(0.2))
model.add(Dense(20, activation="selu", kernel_regularizer=regularizers.l2()))
model.add(Dropout(0.2))
# model.add(Dense(18,activation="selu", kernel_regularizer=regularizers.l2()))
# model.add(Dropout(0.2))

model.add(Dense(1, activation="sigmoid"))

early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
model.compile(optimizer=Adam(learning_rate=0.01),loss='binary_crossentropy',metrics = ['accuracy'])
ann = model.fit(x=x_train, y=y_train, verbose=0, epochs=100, callbacks=[early_stop], validation_split=0.3)

In [ ]:
# Predict prob of y and classes
y_pred_proba = model.predict(x_test)
y_pred = [1 if x>0.5 else 0 for x in y_pred_proba]

# Calculate classification metrics
print(y_pred_proba)
print(y_pred)
display_auc_roc(y_test, y_pred_proba)
display_confusion_matrix(y_test, y_pred, "ANN")
print(classification_report(y_test, y_pred))

#SVM

In [ ]:
 SVM
svm_model = SVC(probability=True)
svm_model.fit(x_train, y_train)
display_triad(svm_model, x_test, y_test, "SVM Confusion Matrix")

# Hyperparameter tuning
from sklearn.model_selection import GridSearchCV
param_grid = {'gamma': [0.01, 0.1, 1, 10, 100], 'C': [0.01, 0.1, 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=3)
grid.fit(x_train, y_train)
print(grid.best_params_)

svm_model_tuned = SVC(**grid.best_params_)
svm_model_tuned.fit(x_train, y_train)
display_triad(svm_model_tuned, x_test, y_test, "SVM Tuned Confusion Matrix")

#RF

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)

# Create a parameter grid
params = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=rf, param_grid=params, cv=3, verbose=2, n_jobs=-1)

grid_search.fit(x_train, y_train)

best_score = grid_search.best_score_

best_params = grid_search.best_params_

print(best_score)
print(best_params)

In [ ]:
classifier = RandomForestClassifier(**best_params)
model_RF = classifier.fit(x_train,y_train)

from sklearn.metrics import classification_report
y_pred = model_RF.predict(x_test)
display_triad(model_RF, x_test, y_test, "Random Forest Classifier")

In [ ]:
feature_importances = pd.Series(classifier.feature_importances_ , index = x_train.columns)
feature_importances.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
import pickle
model_filename = "RandomForest.model"
pickle.dump(model_RF, open(model_filename,'wb'))

from google.colab import files
files.download('RandomForest.model')

#Adaboost

In [ ]:
# HP tuning example for AdaBoost

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

adb_classifier = AdaBoostClassifier()
params = {'learning_rate':[0.2, 0.5, 1.0, 1.5],
          'n_estimators': [25, 50, 100, 150, 200],
          'random_state': [0,2,4]
}

grid_search = GridSearchCV(estimator=adb_classifier, param_grid=params, cv=5, return_train_score = True)
grid_search.fit(x_train, y_train)

# show the best parameters
best_adb_params = grid_search.best_params_

In [ ]:
best_adb_params

In [ ]:
from sklearn.metrics import classification_report
AdaBoost = AdaBoostClassifier(**best_adb_params)

model_AB = AdaBoost.fit(x_train, y_train)
y_pred = model_AB.predict(x_test)
print(classification_report(y_test, y_pred))

In [ ]:
display_triad(model_AB, x_test, y_test, 'AdaBoost Classifier Matrix')